In [8]:
%%file fixtures.py
import pytest

BASE_URL_PETSTORE = 'http://localhost:5296/v2'


@pytest.fixture()
def api_store_order_endpoint():
    return f'{BASE_URL_PETSTORE}/store/order'


@pytest.fixture()
def api_user_endpoint():
    return f'{BASE_URL_PETSTORE}/user'

Overwriting fixtures.py


In [9]:
%%file test_api_user.py
import pytest
import requests
import time
from fixtures import api_user_endpoint

@pytest.mark.parametrize('user,modified_user', [
    ({
         "id": 657911,
         "username": "u9873460111",
         "firstName": "asdf",
         "lastName": "string",
         "email": "string",
         "password": "string",
         "phone": "string",
         "userStatus": 0
     },
     {
         "id": 657911,
         "username": "u9873460111",
         "firstName": "fghh",
         "lastName": "fd",
         "email": "s84",
         "password": "s12",
         "phone": "s000",
         "userStatus": 2
     }),
])
def test_user_api(api_user_endpoint, user: dict, modified_user: dict):
    username = user["username"]

    req_post = requests.post(api_user_endpoint, json=user)
    assert req_post.status_code == 200

    req_get = requests.get(api_user_endpoint + f'/{username}')
    assert req_get.status_code == 200

    get_json = req_get.json()
    for key, val in user.items():
        assert key in get_json
        assert val == get_json[key]

    req_put = requests.put(api_user_endpoint + f'/{username}', json=modified_user)
    assert req_put.status_code == 200

    req_get = requests.get(api_user_endpoint + f'/{username}')
    assert req_get.status_code == 200

    get_json = req_get.json()
    for key, val in modified_user.items():
        assert key in get_json
        assert val == get_json[key]

    req_del = requests.delete(api_user_endpoint + f'/{username}')
    assert req_del.status_code == 200

    req_get = requests.get(api_user_endpoint + f'/{username}')
    assert req_get.status_code == 404



Overwriting test_api_user.py


In [10]:
!python -m pytest test_api_user.py

============================= test session starts =============================
platform win32 -- Python 3.13.1, pytest-9.0.2, pluggy-1.6.0
rootdir: C:\Users\fohtinok\PycharmProjects\PythonProject\4
plugins: allure-pytest-2.15.3, anyio-4.12.1
collected 1 item

test_api_user.py .                                                       [100%]

============================== 1 passed in 0.26s ==============================


## Падает, так как не успевает обработать запросы сама база данных удалённого сервера

In [11]:
%%file test_api_store.py
import pytest
import requests
import time
from fixtures import api_store_order_endpoint

@pytest.mark.parametrize('order', [
    {
        "id": 1239999,
        "petId": 1234,
        "quantity": 0,
        "shipDate": "2025-10-06T12:13:23.238Z",
        "status": "placed",
        "complete": True
    },
    {
        "id": 123590999,
        "petId": 1234999,
        "quantity": 123,
        "shipDate": "2025-10-06T12:13:30.238Z",
        "status": "available",
        "complete": True
    },
])
def test_store_api(api_store_order_endpoint, order):
    orderid = order["id"]

    req_post = requests.post(api_store_order_endpoint, json=order)
    assert req_post.status_code == 200

    req_get = requests.get(api_store_order_endpoint + f'/{orderid}')
    assert req_get.status_code == 200

    get_json = req_get.json()
    for key, val in order.items():
        if key != 'shipDate':
            assert key in get_json
            assert val == get_json[key]

    req_del = requests.delete(api_store_order_endpoint + f'/{orderid}')
    assert req_del.status_code == 200

    req_get = requests.get(api_store_order_endpoint + f'/{orderid}')
    assert req_get.status_code == 404


Overwriting test_api_store.py


In [12]:
!python -m pytest test_api_store.py


============================= test session starts =============================
platform win32 -- Python 3.13.1, pytest-9.0.2, pluggy-1.6.0
rootdir: C:\Users\fohtinok\PycharmProjects\PythonProject\4
plugins: allure-pytest-2.15.3, anyio-4.12.1
collected 2 items

test_api_store.py ..                                                     [100%]

============================== 2 passed in 0.10s ==============================


## Падает, так как не успевает обработать запросы сама база данных удалённого сервера

In [13]:
%%file fixtures.py
import pytest

BASE_URL_PETSTORE = 'https://petstore.swagger.io/v2'


@pytest.fixture()
def api_store_order_endpoint():
    return f'{BASE_URL_PETSTORE}/store/order'


@pytest.fixture()
def api_user_endpoint():
    return f'{BASE_URL_PETSTORE}/user'


Overwriting fixtures.py


In [14]:
%%file models.py
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime
from enum import Enum


class User(BaseModel):
    id: int
    username: str
    firstName: str
    lastName: str
    email: str
    password: str
    phone: str
    userStatus: int = Field(..., description="User Status")


class OrderStatus(str, Enum):
    PLACED = "placed"
    APPROVED = "approved"
    DELIVERED = "delivered"
    AVAILABLE = "available"


class Order(BaseModel):
    id: Optional[int] = None
    petId: Optional[int] = None
    quantity: Optional[int] = None
    shipDate: Optional[datetime] = None
    status: Optional[OrderStatus] = Field(None, description="Order Status")
    complete: Optional[bool] = False




Overwriting models.py
